# Get RGTDB Entries for One User

https://rgtdb.com/user/72763

In [8]:
import pandas as pd
from datetime import datetime as dt
import datetime

dfs = pd.read_html('https://rgtdb.com/user/72763')
df = dfs[1]

## Convert date to datetime, attempt to guess the year

This is going to have troubles around New Year

In [9]:
event_year = str(dt.today().year)
year_subtract = 0
last_month = 13

for index, row in df.iterrows():
    event_month = int(row['Date'][:2])

    if event_month > last_month:
        year_subtract += 1
        event_year = str(dt.today().year - year_subtract)
        print(event_year + '-' + str(event_month))

    df.loc[index, 'Year_Date'] = event_year + '-' + row['Date']

    last_month = event_month
    
df['Year_Date'] = pd.to_datetime(df['Year_Date'], utc=True)
df.set_index('Year_Date', inplace=True)
df

2020-12


,Date,Name
Year_Date,,
2021-03-03 05:00:00+00:00,03-03 05:00,The Recovery Ride
2021-02-28 15:00:00+00:00,02-28 15:00,Lou's Sunday Group Ride
2021-02-28 14:00:00+00:00,02-28 14:00,MRETT14 Lou's Team
2021-02-28 09:00:00+00:00,02-28 09:00,GFNS JENSIE E-FONDO
2021-02-27 16:30:00+00:00,02-27 16:30,Wacky Races 36
...,...,...
2020-11-19 04:45:00+00:00,11-19 04:45,Thursday Sweet Spot AM
2020-10-25 12:30:00+00:00,10-25 12:30,L’ENFER EN FRANCE
2020-10-24 15:00:00+00:00,10-24 15:00,Flemish For Beginners


## Personal Calendar for Signups

g2oh32rptulp8f17e3bdiq9iec@group.calendar.google.com

In [10]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = 'g2oh32rptulp8f17e3bdiq9iec@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


In [11]:
df.index.max() 


Timestamp('2021-03-03 05:00:00+0000', tz='UTC')

## Read Events in Calendar which Are Within Timeframe of RGDB Data

Add one day to timeframe to compensate for unpredictable time zone handling.

Note IDs of existing events so we don't re-create them.

In [12]:
df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    df.loc[(df['Name'] == event.summary) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]

    #if event.start in df.loc[df['Name'] == event.summary].index:
    #    df.at[event.start, 'cal_id'] = event.event_id
    #    df.at[event.start, 'event_obj'] = event
    #    print('found ', event, event.timezone, event.event_id)



In [13]:
df.loc[df['found'] == False]

,Date,Name,cal_id,event_obj,found
Year_Date,,,,,
2021-03-03 05:00:00+00:00,03-03 05:00,The Recovery Ride,None,None,False


In [14]:
for index, row in df.iterrows():

    if row['found'] == False:

        print('* Adding:', index, row['Name'])

        evntColor = '1'

        event = gcEvent(
            str(row['Name']),
            start=index,
            timezone='UTC',
            color = evntColor
        )
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else: 
        print('Skipping', index, row['Name'])

* Adding: 2021-03-03 05:00:00+00:00 The Recovery Ride
ID after add: None Returned event ID: de2g4ml1hi8hvnsdqft37tojq4
Skipping 2021-02-28 15:00:00+00:00 Lou's Sunday Group Ride
Skipping 2021-02-28 14:00:00+00:00 MRETT14 Lou's Team
Skipping 2021-02-28 09:00:00+00:00 GFNS JENSIE E-FONDO
Skipping 2021-02-27 16:30:00+00:00 Wacky Races 36
Skipping 2021-02-27 16:00:00+00:00 Wacky Races 35
Skipping 2021-02-27 15:00:00+00:00 Lou's Saturday Group Ride
Skipping 2021-02-27 13:00:00+00:00 Wacky Races 29
Skipping 2021-02-27 11:30:00+00:00 Wacky Races 26
Skipping 2021-02-27 10:30:00+00:00 Wacky Races 24
Skipping 2021-02-27 10:00:00+00:00 Wacky Races 23
Skipping 2021-02-27 08:00:00+00:00 Wacky Races 19
Skipping 2021-02-26 10:30:00+00:00 KISS Racing Team Crit
Skipping 2021-02-25 15:00:00+00:00 Chat Laps
Skipping 2021-02-25 10:50:00+00:00 KISS at Base
Skipping 2021-02-25 07:00:00+00:00 Breakfast Club
Skipping 2021-02-24 05:00:00+00:00 The Recovery Ride
Skipping 2021-02-21 15:00:00+00:00 Lou's Sunday G